In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


In [2]:

# Initialize the Spark session
spark = SparkSession.builder \
    .appName("Fashion Dataset Cleaning") \
    .getOrCreate()


25/04/16 01:00:44 WARN Utils: Your hostname, indrani resolves to a loopback address: 127.0.1.1; using 192.168.0.107 instead (on interface wlo1)
25/04/16 01:00:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/16 01:00:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:

# Load the CSV data
df = spark.read.csv("FashionDataset.csv", header=True, inferSchema=True)


In [4]:

# Show the first few rows of the dataset
df.show(5)


25/04/16 01:00:56 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


+-----+---------+--------------------+--------------------+----+---------+--------+--------+
|  _c0|BrandName|             Deatils|               Sizes| MRP|SellPrice|Discount|Category|
+-----+---------+--------------------+--------------------+----+---------+--------+--------+
|    0|     life|solid cotton blen...|Size:Large,Medium...|  Rs|     NULL|    NULL|    NULL|
|1699"|      849|             50% off|   Westernwear-Women|NULL|     NULL|    NULL|    NULL|
|    1|     only|polyester peter p...|    Size:34,36,38,40|  Rs|     NULL|    NULL|    NULL|
|3499"|     2449|             30% off|   Westernwear-Women|NULL|     NULL|    NULL|    NULL|
|    2|  fratini|solid polyester b...|Size:Large,X-Larg...|  Rs|     NULL|    NULL|    NULL|
+-----+---------+--------------------+--------------------+----+---------+--------+--------+
only showing top 5 rows



In [5]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- BrandName: string (nullable = true)
 |-- Deatils: string (nullable = true)
 |-- Sizes: string (nullable = true)
 |-- MRP: string (nullable = true)
 |-- SellPrice: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Category: string (nullable = true)



25/04/16 01:00:59 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## Remove missing data

In [14]:
df_cleaned = df.dropna(how='any')

## Drop Duplicates

In [15]:
df_cleaned = df_cleaned.dropDuplicates()

In [16]:
df_cleaned.count()

25/04/16 01:03:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


7014

## Distict values

In [17]:
distinct_counts = df_cleaned.groupBy("Category").count()
distinct_counts.show()

25/04/16 01:03:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


+--------------------+-----+
|            Category|count|
+--------------------+-----+
|      Footwear-Women|  834|
|Lingerie&Nightwea...| 1133|
|   Westernwear-Women| 1696|
|     Fragrance-Women|  289|
|     Jewellery-Women|  635|
|       Watches-Women|  750|
|    Indianwear-Women| 1677|
+--------------------+-----+



## Stop spark

In [18]:
df_cleaned.show(50, False)

25/04/16 01:03:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


+---+-----------------+---------------------------------------------------------------------+----------------------------------------------------------------+---+---------+--------+------------------------+
|_c0|BrandName        |Deatils                                                              |Sizes                                                           |MRP|SellPrice|Discount|Category                |
+---+-----------------+---------------------------------------------------------------------+----------------------------------------------------------------+---+---------+--------+------------------------+
|24 |levis            |skinny fit regular length cotton womens jeans - dk indigo            |Size:28,30,32,34                                                |Nan|3699     |Nan     |Westernwear-Women       |
|18 |campus sutra     |solid cotton hood womens co-ordinate set - pink                      |Size:Large,Medium,Small,X-Large                                 |Nan|1599     |

In [19]:
df_cleaned = df_cleaned.drop("_c0", "MRP", "Discount")

In [20]:
df_cleaned = df_cleaned.withColumnRenamed("Deatils", "Details")

In [12]:
df_cleaned.printSchema()

root
 |-- BrandName: string (nullable = true)
 |-- Details: string (nullable = true)
 |-- Sizes: string (nullable = true)
 |-- SellPrice: string (nullable = true)
 |-- Category: string (nullable = true)



In [21]:
df_converted = df_cleaned.withColumn("SellPrice", col("SellPrice").cast("int"))

In [22]:
df = df_converted

In [26]:
from pyspark.sql.functions import col, isnan
df = df.filter(~isnan(col("Sizes")))
df.show(50, False)

25/04/16 01:07:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


+-----------------------+------------------------------------------------------------------------------------------------+----------------------------------------------------------------+---------+------------------------+
|BrandName              |Details                                                                                         |Sizes                                                           |SellPrice|Category                |
+-----------------------+------------------------------------------------------------------------------------------------+----------------------------------------------------------------+---------+------------------------+
|levis                  |skinny fit regular length cotton womens jeans - dk indigo                                       |Size:28,30,32,34                                                |3699     |Westernwear-Women       |
|campus sutra           |solid cotton hood womens co-ordinate set - pink                                    

In [27]:
# Filter out rows where 'category' is None or null
df = df.filter(col("Category").isNotNull())


In [61]:
df.count()

25/04/16 00:36:56 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


7014

In [28]:

# Get the total number of records
total_records = df.count()

# Number of records you want to pick
sample_size = 1000

# Get the count of records per category
category_counts = df.groupBy("Category").count().collect()


25/04/16 01:08:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv
25/04/16 01:08:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


In [29]:
# Calculate the fraction for each category based on its proportion in the total dataset
fractions = {}
for row in category_counts:
    category = row['Category']
    count = row['count']
    # Calculate the fraction based on the category size
    fractions[category] = (count / total_records) * sample_size / count

# Perform stratified sampling
df_sampled = df.sampleBy("Category", fractions, seed=42)

In [65]:
df_sampled.count()

25/04/16 00:37:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


1049

In [30]:
df_sampled.write.json("output", mode="overwrite")

25/04/16 01:08:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


In [70]:
df_sampled.groupBy("Category").count().show()

25/04/16 00:47:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
 Schema: _c0, BrandName, Deatils, Sizes, MRP, SellPrice, Discount, Category
Expected: _c0 but found: 
CSV file: file:///home/alexpy/Desktop/poridhi.io-ai-hackathon/data_processor/FashionDataset.csv


+--------------------+-----+
|            Category|count|
+--------------------+-----+
|      Footwear-Women|  119|
|Lingerie&Nightwea...|  168|
|   Westernwear-Women|  264|
|     Fragrance-Women|   44|
|     Jewellery-Women|   86|
|       Watches-Women|  111|
|    Indianwear-Women|  257|
+--------------------+-----+



In [31]:
spark.stop()